<a href="https://colab.research.google.com/github/hchung596/dArwin_TrashNet/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Dependencies to be able to load the model and show a picture 
import cv2 
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import PIL

In [ ]:
#Image capture functions 
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=1.0):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))

  filename=cap(data)

  return filename

In [ ]:
#Helper function 
def js_to_image(js_reply):
  image_bytes = b64decode(js_reply.split(',')[1])
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  img = cv2.imdecode(jpg_as_np, flags=1)
  return img

In [ ]:
#Helper function
def cap(data):
  frame = js_to_image(data)
  rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  filename='photo.jpg'
  cv2.imwrite(filename, frame)
  return filename  

In [ ]:
#STEP 1: Place .h5 file into runtime directory 
#STEP 2: Place an image of plastic/metal into runtime directory 
model = keras.models.load_model("MODEL5.h5") #Loads .h5 file in the runtime 
obj = cv2.imread(take_photo()) #File name for your image, make sure file type and name match
obj = cv2.cvtColor(obj, cv2.COLOR_RGB2BGR) #Changes the color values 
obj = tf.image.resize(obj, (224,224)) #Resizes image
plt.imshow(obj.numpy().astype(int)) #Turns it into a numpy array (to be able to show)
plt.show #Shows image in output
expand = np.expand_dims(obj,0) #Adds a dimension to the image to be able to feed it to model
predict1 = model.predict(expand) #Tells model to make a prediction 
classes = ['cardboard', 'food', 'glass', 'metal', 'newhands', 'paper', 'plastic'] #Classes 
print("IMG IS", classes[np.argmax(predict1)])

OSError: ignored